# Retour sur la session 2 

In [6]:
import numpy as np
import pandas as pd

geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';')

# Utilisation d'environnements virtuels 
création d'un environnement par Projet PYTHON

Utilisation d'environnements virtuels :

Pour isoler les librairies utilisées dans un projet, se créer un environnement Python par projet, par exemple :

conda create --name myenv

Ensuite activer l'environnement à utiliser dans un terminal au niveau de votre notebook :

conda activate myenv (linux) ou activate myenv (Windows)

Puis lancer jupyter :

jupyter notebook


Ou passer par Anaconda Navigator >> environments >> Nouveau

que signifie "isoler les librairies" ? si travail dans différents projets, utilisation de Pandas pour le Projet 1. possible que le projet fonctionne avec les librairies du temps t et ensuite projet2 avec autre exemple.

# Utilisation de Git

Utilisation de git

Se créer un compte sur github et le mettre dans l'organisation du Master
Rappatrier le cours une première fois avec : git clone https://github.com/fran6w/MDI721
Puis le mettre à jour à chaque fois avec : git fetch
Copier ce repository local dans un répertoire de travail pour suivre le cours ou faire les projets.
Copier ensuite vos travaux dans le repository local de votre compte github et les exporter avec la séquence :
git add .
git commit -m "message"
git push origine

Tri 

sort_values()
triage selon plusieurs colonnes 
ordre lexicographique => première colonne triée et si égalité, utilisation de la seconde colonne.

In [ ]:
#Pour accéder aux différentes colonnes:
geo['Commune'] #1ere forme
geo.Commune #equivalent
#mais 1ere forme à privilégier 
#difficulté pour récupérer les données loc/iloc

#conversion des objets 
geo["Altitude Moyenne"] = geo["Altitude Moyenne"].astype(int)

#astype(int) est une méthode de series 
#int(valeur) est une fonction qui convertit une valeur en entier


In [ ]:
#Conversion en colonne catégorie
#possibilité de créer des catégories 

#classe CategoricalDtype #usage pour créer un type catégoriel

geo["Statut"]. value_counts() # donne les chaines de caractères en ordre alphabétique
#Mise en place d'un attribut catégoriel

#faire la liste des catégories


statuts = ["Commune simple", "Chef-lieu canton", "Sous-préfecture",
            "Préfecture", "Préfecture de région", "Capitale d'état"]

cat_statut = CategoricalDtype(categories=statuts, ordered=True)

## pour gagner en espace mémoire
#dans le 1er cas autant de chaines de caractères que d'espaces mémoires utilisés
#pour les catégories en mémoire sur un plus petit espace

# Correction Projets 

In [1]:
#Possibilité de combiner deux séries 
#.combine

#Pour avoir les commandes magiques
%magic

In [2]:
%who

Interactive namespace is empty.


In [ ]:
geo['Cp_Ville'] = geo['Code Postal'].combien(geo['Commune'], lambda x, y=f{x}{y})
#mais prend beaucoup de temps  mieux vaut utiliser :
geo['Cp_Ville'] = geo['Code Postal'] + ' ' + geo['Commune']

In [ ]:
#création de la fonction ville
def ville(lat, long):
    dist = np.sqrt(((geo['Latitude']-lat)**2) + (geo['Longitute'] - long)**2)
    index = dist.idxmin()
    return geo.loc[index, 'CP Ville']

#ici pas besoin d'avoir le carré, car on ne demande que l'index min et pas le calcul exact
#pour faire de l'optimisation, pas besoin du carré. 

In [8]:
#Projet 2 
geo['geo_shape'].values[0]
import json
type(json.loads(geo['geo_shape'].values[0]))

dict

In [9]:
geo_shape_json = geo['geo_shape'].apply(json.loads)
#s'applique à tous 

In [10]:
geo_shape_json.head()

0    {'type': 'Polygon', 'coordinates': [[[0.671852...
1    {'type': 'Polygon', 'coordinates': [[[2.906996...
2    {'type': 'Polygon', 'coordinates': [[[4.091674...
3    {'type': 'Polygon', 'coordinates': [[[1.829610...
4    {'type': 'Polygon', 'coordinates': [[[4.927176...
Name: geo_shape, dtype: object

In [11]:
geo_shape_json.apply(lambda x: len(x['coordinates'])).value_counts()

1    36660
2       80
4        1
3        1
Name: geo_shape, dtype: int64

In [14]:
# les communes les plus complexes correspondent à celle qui ont le plus de polygones ici qui a  polygone
geo.loc[geo_shape_json.apply(lambda x : len(x['coordinates'])) == 4, 'Commune']




28831    HYERES
Name: Commune, dtype: object

In [15]:
# la seconde commune la plus complexe avec 3 polygones
geo.loc[geo_shape_json.apply(lambda x : len(x['coordinates'])) == 3, 'Commune']

23622    VAL-DE-REUIL
Name: Commune, dtype: object

In [18]:
#on cherche les communes complexes avec plusieurs polygones

geo.loc[(geo_shape_json.apply(lambda x: x['type'])=='Polygon') & (geo_shape_json.apply(lambda x : len(x['coordinates'])) == 2, 'Commune')]

ValueError: setting an array element with a sequence.

In [19]:
from shapely.geometry import Polygon

In [22]:
def est_enclave (coordinates):
    if len(coordinates) ==2:
        return Polygon(coordinates[0]).contains(Polygon(coordinates[1]))
    
    return False

#a reprendre
geo_shape_json.apply(lambda x : est_enclave(x['coordinates']))

ValueError: A LinearRing must have at least 3 coordinate tuples